In [1]:
import os

from keyname import keyname as kn
import matplotlib.pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import os
import pandas as pd
import seaborn as sns
from teeplot import teeplot


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: d4b32041fe1f
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 2
nbname: reconstruction-quality-summary-statistics
nbpath: /opt/hstrat-evolutionary-inference/binder/reconstruction-quality-summary-statistics.ipynb
revision: null
session: e9931d90-c15f-43df-ba34-d95054e1070e
timestamp: 2023-03-26T01:04:23Z00:00


IPython==7.16.1
keyname==0.5.2
yaml==6.0
matplotlib==3.7.1
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
seaborn==0.11.0
teeplot==0.5.0
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.0.3
json==2.0.9
six==1.16.0
ipykernel==5.5.3


In [3]:
df = pd.read_csv("https://osf.io/ck47r/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


6f7ca67df987a1b8


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,subsampling-fraction,tournament_size,treatment,trie-postprocess,_generation,_index,ext,_,triplet_distance,quartet_distance
0,reconstructed-tree,0,np.random.standard_normal,32768,1,8,0.01,3.051758e-08,32768,29,...,0.01,2,20,naive,32768,832,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.098844e-14,-2.098824e-14
1,reconstructed-tree,0,np.random.standard_normal,32768,1,8,0.01,3.051758e-08,32768,6,...,0.01,2,20,naive,32768,188,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.111658e-14,-2.111638e-14
2,reconstructed-tree,0,np.random.standard_normal,32768,1,8,0.01,3.051758e-08,32768,34,...,1.00,2,20,rollback,32768,972,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.088371e-14,-2.088351e-14
3,reconstructed-tree,0,np.random.standard_normal,32768,1,8,0.01,3.051758e-08,32768,17,...,1.00,2,20,expected,32768,496,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,1.428246e-03,5.220629e-03
4,reconstructed-tree,0,np.random.standard_normal,32768,1,8,0.01,3.051758e-08,32768,49,...,0.01,2,20,rollback,32768,1392,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.100494e-14,-2.100474e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100795,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,44,...,1.00,2,25,naive,262144,1257,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,6.865545e-04,2.452341e-03
100796,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,47,...,1.00,2,25,rollback,262144,1341,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.097384e-14,-2.097364e-14
100797,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,19,...,0.01,2,25,expected,262144,557,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,-2.106571e-14,-2.106551e-14
100798,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-06,32768,6,...,1.00,2,25,rollback,262144,193,.csv.gz,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,5.161053e-04,1.816271e-03


In [4]:
evolutionary_variables = [
    "mut_distn",  # sensitivity analysis
    "num_generations",  # sensitivity analysis
    "num_islands",
    "num_niches",
    "p_island_migration",  # sensitivity analysis
    "p_niche_invasion",  # sensitivity analysis
    "population_size",  # doesn't change
    "tournament_size",
]


In [5]:
df = df[
    (df["subsampling-fraction"] == 1.0) & (df["trie-postprocess"] == "naive")
].copy()


In [6]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2, 3.0517578125e-08): "plain",
        (1, 1, 1, 3.0517578125e-08): "neutral selection",
        (1, 1, 8, 3.0517578125e-08): "strong selection",
        (1, 4, 2, 3.0517578125e-06): "weak 4 niche ecology",
        (1, 4, 2, 3.0517578125e-08): "4 niche ecology",
        (1, 8, 2, 3.0517578125e-08): "8 niche ecology",
        (1024, 1, 2, 3.0517578125e-08): "spatial structure",
    }.get(
        tuple(
            row[
                [
                    "num_islands",
                    "num_niches",
                    "tournament_size",
                    "p_niche_invasion",
                ]
            ]
        ),
        np.nan,  # default
    ),
    axis="columns",
)


In [7]:
df["quality"] = df.apply(
    lambda row: {
        ("reconstructed-tree", 3.0): "33% resolution",
        ("reconstructed-tree", 10.0): "10% resolution",
        ("reconstructed-tree", 30.0): "3% resolution",
        ("reconstructed-tree", 100.0): "1% resolution",
        ("collapsed-phylogeny", 0): "perfect resolution",
    }.get(
        tuple(row[["a", "resolution"]].fillna(0)),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime", "quality"]).copy()
df["quality"].unique()


array(['33% resolution', '10% resolution', '1% resolution',
       '3% resolution'], dtype=object)

In [8]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]

df["sensitivity_analysis_variables"] = df[
    sensitivity_analysis_variables
].apply(kn.pack, axis=1)


In [9]:
stats = df.groupby(
    ["regime", "quality", "sensitivity_analysis_variables"], as_index=False
).agg(
    {
        "triplet_distance": ["mean", "median", "std", "max"],
    }
)
stats.columns = stats.columns.map(' '.join)
with pd.option_context("display.max_colwidth", None, "display.max_rows", None):
    display(stats)


,regime,quality,sensitivity_analysis_variables,triplet_distance mean,triplet_distance median,triplet_distance std,triplet_distance max
0,4 niche ecology,1% resolution,epoch=0+mut_distn=np.random.exponential,2.555102e-06,-2.099832e-14,1.051878e-05,6.512716e-05
1,4 niche ecology,1% resolution,epoch=0+mut_distn=np.random.standard_normal,1.296532e-06,-2.087247e-14,4.620749e-06,2.405889e-05
2,4 niche ecology,1% resolution,epoch=2+mut_distn=np.random.exponential,-2.110073e-14,-2.111276e-14,1.398730e-16,-2.084410e-14
3,4 niche ecology,1% resolution,epoch=2+mut_distn=np.random.standard_normal,-2.106797e-14,-2.104146e-14,1.332805e-16,-2.084142e-14
4,4 niche ecology,1% resolution,epoch=7+mut_distn=np.random.exponential,-2.106089e-14,-2.106063e-14,1.283913e-16,-2.082979e-14
5,4 niche ecology,1% resolution,epoch=7+mut_distn=np.random.standard_normal,-2.107361e-14,-2.104674e-14,1.545536e-16,-2.082827e-14
6,4 niche ecology,10% resolution,epoch=0+mut_distn=np.random.exponential,3.985577e-04,-2.096824e-14,9.912286e-04,4.156146e-03
7,4 niche ecology,10% resolution,epoch=0+mut_distn=np.random.standard_normal,8.851201e-04,-2.089465e-14,1.297112e-03,5.563309e-03
8,4 niche ecology,10% resolution,epoch=2+mut_distn=np.random.exponential,-2.105604e-14,-2.105831e-14,1.381118e-16,-2.082909e-14
9,4 niche ecology,10% resolution,epoch=2+mut_distn=np.random.standard_normal,3.896025e-05,-2.100538e-14,2.754906e-04,1.948013e-03


In [10]:
os.makedirs("outdata", exist_ok=True)
stats.to_csv(
    "outdata/a=tree-reconstruction-quality-triplet-summary-statistics.csv", index=False
)


In [11]:
stats = df.groupby(
    ["regime", "quality", "sensitivity_analysis_variables"], as_index=False
).agg(
    {
        "quartet_distance": ["mean", "median", "std", "max"],
    }
)
stats.columns = stats.columns.map(' '.join)
with pd.option_context("display.max_colwidth", None, "display.max_rows", None):
    display(stats)


,regime,quality,sensitivity_analysis_variables,quartet_distance mean,quartet_distance median,quartet_distance std,quartet_distance max
0,4 niche ecology,1% resolution,epoch=0+mut_distn=np.random.exponential,9.181717e-06,-2.099812e-14,3.692157e-05,2.243568e-04
1,4 niche ecology,1% resolution,epoch=0+mut_distn=np.random.standard_normal,4.863782e-06,-2.087227e-14,1.762516e-05,9.237605e-05
2,4 niche ecology,1% resolution,epoch=2+mut_distn=np.random.exponential,-2.110053e-14,-2.111255e-14,1.398703e-16,-2.084390e-14
3,4 niche ecology,1% resolution,epoch=2+mut_distn=np.random.standard_normal,-2.106776e-14,-2.104126e-14,1.332780e-16,-2.084122e-14
4,4 niche ecology,1% resolution,epoch=7+mut_distn=np.random.exponential,-2.106069e-14,-2.106043e-14,1.283889e-16,-2.082959e-14
5,4 niche ecology,1% resolution,epoch=7+mut_distn=np.random.standard_normal,-2.107340e-14,-2.104654e-14,1.545506e-16,-2.082807e-14
6,4 niche ecology,10% resolution,epoch=0+mut_distn=np.random.exponential,1.279608e-03,-2.096804e-14,3.175339e-03,1.324154e-02
7,4 niche ecology,10% resolution,epoch=0+mut_distn=np.random.standard_normal,2.834499e-03,-2.089445e-14,4.112612e-03,1.758798e-02
8,4 niche ecology,10% resolution,epoch=2+mut_distn=np.random.exponential,-2.105583e-14,-2.105811e-14,1.381091e-16,-2.082889e-14
9,4 niche ecology,10% resolution,epoch=2+mut_distn=np.random.standard_normal,-2.103882e-14,-2.100518e-14,1.420209e-16,-2.083775e-14


In [12]:
os.makedirs("outdata", exist_ok=True)
stats.to_csv(
    "outdata/a=tree-reconstruction-quality-quartet-summary-statistics.csv", index=False
)
